<a href="https://colab.research.google.com/github/LUCY-JungHwaKim/DataAnalysis/blob/main/%EA%B8%B0%EC%B6%9C2%ED%9A%8C_%EC%9E%91%EC%97%85%EC%A0%9C2%EC%9C%A0%ED%98%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/AnalyticsKnight/yemoonsaBigdata

Cloning into 'yemoonsaBigdata'...
remote: Enumerating objects: 485, done.
remote: Counting objects: 100% (122/122), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 485 (delta 66), reused 49 (delta 44), pack-reused 363
Receiving objects: 100% (485/485), 8.89 MiB | 15.99 MiB/s, done.
Resolving deltas: 100% (251/251), done.


In [9]:
import pandas as pd
X_train = pd.read_csv('./yemoonsaBigdata/datasets/Part3/204_x_train.csv')
X_test = pd.read_csv('./yemoonsaBigdata/datasets/Part3/204_x_test.csv')
y_train = pd.read_csv('./yemoonsaBigdata/datasets/Part3/204_y_train.csv')

In [11]:
y_train.head()

,ID,Reached.on.Time_Y.N
0,9902,0
1,9501,0
2,6111,1
3,588,1
4,10806,0


In [12]:
X_data = pd.concat([X_train, X_test])

In [15]:
X_data.iloc[:len(X_train)]

,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms
0,9902,F,Ship,3,5,214,2,medium,F,6,4578
1,9501,A,Ship,4,2,201,3,medium,F,4,4613
2,6111,A,Ship,4,2,264,3,low,M,10,5137
3,588,F,Ship,3,5,194,2,medium,M,4,3496
4,10806,F,Road,5,5,269,5,medium,F,3,1120
...,...,...,...,...,...,...,...,...,...,...,...
6594,3086,F,Ship,4,1,213,3,low,F,21,3585
6595,7154,F,Ship,5,1,257,5,low,M,9,1329
6596,4897,D,Ship,3,1,264,4,low,M,10,4265
6597,10553,C,Flight,3,1,185,2,medium,F,9,4316


In [21]:
COL_CAT = ['Warehouse_block', 'Mode_of_Shipment', 'Customer_rating', 'Product_importance', 'Gender']
from sklearn.preprocessing import LabelEncoder

lhe = LabelEncoder()
for col_ in COL_CAT:
  lhe.fit(X_data[col_])
  X_data[col_] = lhe.transform(X_data[col_])

### 학습데이터 할때는 fit, transform
### 테스트 데이터 할때는 그냥 transform

In [22]:
X_trains = X_data.iloc[:len(X_train)]
X_tests = X_data.iloc[len(X_train):]

In [23]:
X_trains.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6599 entries, 0 to 6598
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype
---  ------               --------------  -----
 0   ID                   6599 non-null   int64
 1   Warehouse_block      6599 non-null   int64
 2   Mode_of_Shipment     6599 non-null   int64
 3   Customer_care_calls  6599 non-null   int64
 4   Customer_rating      6599 non-null   int64
 5   Cost_of_the_Product  6599 non-null   int64
 6   Prior_purchases      6599 non-null   int64
 7   Product_importance   6599 non-null   int64
 8   Gender               6599 non-null   int64
 9   Discount_offered     6599 non-null   int64
 10  Weight_in_gms        6599 non-null   int64
dtypes: int64(11)
memory usage: 618.7 KB


In [43]:
X_trains = X_trains.iloc[:, 1:]
X_tests = X_tests.iloc[:, 1:]
## ID 컬럼 삭제

In [44]:
## scaler 실행 (전부다 수치형으로 바꼈기 때문에 scaler 진행해도 됨)
from sklearn.preprocessing import MinMaxScaler

scl = MinMaxScaler()
scl.fit(X_trains) # fit하고 transform 두번해야함
X_train_scl = scl.transform(X_trains)
X_test_scl = scl.transform(X_tests)

In [46]:
y_train = y_train.iloc[:, 1:]

In [47]:
from sklearn.model_selection import train_test_split

X_tr, X_val, y_tr, y_val = train_test_split(X_train_scl, y_train, test_size=0.3)

In [48]:
y_tr.shape

(4619, 1)

In [49]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier

In [63]:
from sklearn.metrics import roc_auc_score

for i in range(1,10):
  model_rf = RandomForestClassifier(n_estimators=i)
  model_rf.fit(X_tr, y_tr)
  y_pred_val = model_rf.predict_proba(X_val)[:,1]
  print("model_rf + n_estimators" + str(i))
  print(roc_auc_score(y_val, y_pred_val)) ## 정답ㄷ이 앞, 예측이 뒤

  
for i in range(2,7):
  model_knn = KNeighborsClassifier(n_neighbors=i) # 인자 기억
  model_knn.fit(X_tr, y_tr)
  y_pred_val = model_knn.predict_proba(X_val)[:,1]
  print("model_knn + neighbors" + str(i))
  print(roc_auc_score(y_val, y_pred_val)) ## 정답ㄷ이 앞, 예측이 뒤

model_xgb = XGBClassifier()
model_xgb.fit(X_tr, y_tr)
y_pred_val = model_xgb.predict_proba(X_val)[:, 1]
print("model_xgb")
print(roc_auc_score(y_val, y_pred_val)) ## 정답ㄷ이 앞, 예측이 뒤

  
for i in range(2,7):
  model_dt = DecisionTreeClassifier(max_depth=i) # 인자 기억
  model_dt.fit(X_tr, y_tr)
  y_pred_val = model_dt.predict_proba(X_val)[:,1]
  print("model_dt + depth" + str(i))
  print(roc_auc_score(y_val, y_pred_val)) ## 정답ㄷ이 앞, 예측이 뒤

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was pas

model_rf + n_estimators1
0.6019659377628259
model_rf + n_estimators2
0.6640874684608915
model_rf + n_estimators3
0.7080498317914213
model_rf + n_estimators4
0.7140012615643397
model_rf + n_estimators5
0.7334419680403701
model_rf + n_estimators6
0.7327097350714885
model_rf + n_estimators7
0.7425988225399495
model_rf + n_estimators8
0.7350467830109336
model_rf + n_estimators9
0.7475236543313708


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


model_knn + neighbors2
0.6857201429772919
model_knn + neighbors3
0.695561921783011


/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


model_knn + neighbors4
0.7127344407064762
model_knn + neighbors5
0.7204468040370059


/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


model_knn + neighbors6
0.7227086837678722


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


model_xgb
0.7550662321278385
model_dt + depth2
0.7462573591253154
model_dt + depth3
0.7538067703952902
model_dt + depth4
0.7575094617325483
model_dt + depth5
0.7552544154751892
model_dt + depth6
0.7438120269133727


In [66]:
best_model = DecisionTreeClassifier(max_depth = 5)
best_model.fit(X_tr, y_tr)
print(best_model.score(X_tr, y_tr))
y_test_pred = best_model.predict_proba(X_test_scl)[:, 1]
data = pd.DataFrame({'y_pred':y_test_pred})
data

0.688027711625893


,y_pred
0,0.380952
1,0.468750
2,0.457182
3,0.380952
4,0.457182
...,...
4395,0.380952
4396,0.380952
4397,0.453711
4398,0.380952


In [ ]:
# data.to_csv('./yemoonsaB')